In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [83]:
import gensim
model = gensim.models.Word2Vec.load('/content/drive/My Drive/ja/ja.bin')
word_vectors = model.wv

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [85]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [86]:
# ファイル名を指定してシートを開く
sht = gc.open('COVID19_word_num_sheet')
worksheet = sht.get_worksheet(0)

# 読み込むセルの範囲の指定(読み込みたい範囲がわかっているのであれば適宜書き換える)
row_cnt = worksheet.row_count
col_cnt = 2

cells = worksheet.range(1, 1, row_cnt, col_cnt)

table_data = []
cols = []

for i, cell in enumerate(cells):
  cols.append(cell.value)
  if (i + 1) % col_cnt == 0:
    table_data.append(cols)
    cols = []

print(table_data)

[['コロナ', '12223'], ['ウイルス', '11271'], ['新型', '8738'], ['感染', '6827'], ['対策', '1672'], ['月', '1410'], ['確認', '1336'], ['拡大', '928'], ['影響', '865'], ['発表', '792'], ['人', '647'], ['防止', '622'], ['検査', '611'], ['マスク', '598'], ['県内', '558'], ['情報', '543'], ['ニュース', '538'], ['患者', '534'], ['状況', '509'], ['発生', '479'], ['予防', '473'], ['質問', '468'], ['台風', '466'], ['世界', '437'], ['お願い', '423'], ['～', '384'], ['開催', '373'], ['死亡', '369'], ['ワクチン', '359'], ['陽性', '349'], ['支援', '335'], ['消毒', '312'], ['医療', '302'], ['避難', '302'], ['中止', '291'], ['対応', '288'], ['関連', '276'], ['男性', '276'], ['効果', '267'], ['実施', '264'], ['接触', '263'], ['市内', '263'], ['営業', '261'], ['新規', '259'], ['着用', '253'], ['お知らせ', '248'], ['終息', '248'], ['募集', '239'], ['症状', '237'], ['死者', '234'], ['女性', '229'], ['リンク', '223'], ['インフルエンザ', '215'], ['生活', '207'], ['時点', '206'], ['流行', '199'], ['協力', '199'], ['タ', '197'], ['グ', '196'], ['経済', '193'], ['全国', '189'], ['サイト', '189'], ['匿名', '188'], ['更新', '185'], ['利用', '182'], ['

In [97]:
from sklearn.decomposition import PCA

length = len(table_data)
exist_word = []
word_num = []
data = []
j = 0
while j < length:
  if table_data[j][0] in word_vectors.vocab:
    exist_word.append(table_data[j][0])
    word_num.append(table_data[j][1])
    data.append(model[table_data[j][0]])
    j += 1
  else:
    j += 1
 
pca = PCA(n_components=3)
pca.fit(data)
data_pca= pca.transform(data)

length_data = len(data_pca)

data_x = []
data_y = []
data_z = []
i = 0
while i < length_data:
    #配列形式に整形
    data_x.append(data_pca[i][0])
    data_y.append(data_pca[i][1])
    data_z.append(data_pca[i][2])
    i += 1

output = []
output.append(["word", "num", "x", "y", "z"])
for i in range(len(exist_word)):
  output_list = []

  output_list.append(exist_word[i])
  output_list.append(word_num[i])
  output_list.append(data_x[i])
  output_list.append(data_y[i])
  output_list.append(data_z[i])
  output.append(output_list)

print(output)

[['word', 'num', 'x', 'y', 'z'], ['コロナ', '12223', 0.846327989484844, -0.9343290666632627, 1.9297074196458674], ['ウイルス', '11271', -5.9864480451326765, -5.1550360035399585, 1.229884060183547], ['新型', '8738', -3.72917344963716, -0.5887894897285413, 4.297111221622185], ['感染', '6827', -2.614701852844208, -0.7579217504386815, -1.6535308961589372], ['月', '1410', 8.804436757170217, 8.216181737815978, 1.08219183824793], ['確認', '1336', -9.885547074092507, 1.6278839757497594, -1.6675008903458268], ['影響', '865', -5.371965802743982, -3.0088819612075537, -2.1984996056286863], ['人', '647', 2.6116659136234417, -4.886649588261438, -7.084809599072392], ['防止', '622', -6.536220410207948, -0.08273060171670132, 5.440717365801413], ['マスク', '598', -1.9425481432039091, -6.457744666999252, 1.6005817008879037], ['情報', '543', -5.090541544083638, 2.2527214486997362, 3.060288236589277], ['ニュース', '538', -0.9212399292456873, 0.32591173641909, 0.5477809040969401], ['患者', '534', -5.747085388549374, -4.067122044981984, 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [98]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [99]:
row_len = len(output)
col_len = len(output[0])

sheet_name = "COVID19_word2vec_sheet"

sh = gc.create(sheet_name)
worksheet = gc.open(sheet_name).sheet1

worksheet.add_rows(row_len)

cell_list = worksheet.range(1, 1, row_len, col_len)
col_list = [flatten for inner in output for flatten in inner]

for cell, col in zip(cell_list, col_list):
  cell.value = col

worksheet.update_cells(cell_list)

{'spreadsheetId': '1pQaerSmxybyKihM6KU_OLgZr2Czj26_O1CPmGSYDx3c',
 'updatedCells': 27050,
 'updatedColumns': 5,
 'updatedRange': "'シート1'!A1:E5410",
 'updatedRows': 5410}